Databricks Asset Bundles must be deployed using the Databricks CLI, not from the Databricks UI.

Databricks UI doesnot suppport deployment

The UI Deploy button only syncs notebooks/files, it does not deploy infrastructure resources like DLT pipelines or jobs.

In [0]:
import dlt
from pyspark.sql.functions import current_timestamp, col

SOURCE_TABLE = "source.default.orders"

BRONZE_CATALOG = "bronze"
BRONZE_SCHEMA  = "default" #Any have anything based on business use case it is an testing schema.
BRONZE_TABLE   = "orders"

@dlt.table( #Delta Live Tables decorator: Creates and manages a table in Catalog
    name=f"{BRONZE_CATALOG}.{BRONZE_SCHEMA}.{BRONZE_TABLE}",
    comment="Bronze ingestion from source.default.orders (raw copy) with bronze_ingestion_date."
)
def bronze_orders():
    return (
        spark.table(SOURCE_TABLE) #Reads Source table
        .select(
            col("id").cast("int").alias("id"),
            col("category").cast("string").alias("category"),
            col("amount").cast("int").alias("amount"),
            col("ingestion_date").cast("timestamp").alias("ingestion_date")  # from source
        )
        .withColumn("bronze_ingestion_date", current_timestamp())            # added by bronze
    )

#df.write.format("delta").mode("append").saveAsTable("bronze.default.orders") {Works in Job} - NOT REQUIRED IN DLT, as dlt decorater handles it.

In [0]:
from pyspark.sql.functions import current_timestamp, col

df = (
    spark.table("source.default.orders")
    .select(
        col("id").cast("int").alias("id"),
        col("category").cast("string").alias("category"),
        col("amount").cast("int").alias("amount"),
        col("ingestion_date").cast("timestamp").alias("ingestion_date")
    )
    .withColumn("bronze_ingestion_date", current_timestamp())
)

df.write.format("delta").mode("append").saveAsTable("workspace.default.bronze_orders")
